# Regresja i klasyfikacja

## Miłosz Mizak

### 1. Wstęp
Przedmiotem czwartego zadania było zadanie klasyfikacji. Należało zaimplementować algorytm SVM wraz z implementacją jąder, a następnie zbadać działanie algorytmu na podstawie [Wine Quality Data Set](https://archive.ics.uci.edu/ml/datasets/wine+quality). Aby tego dokonać, postanowiłem użyć algorytmu do przewidywania jakości danego wina na podstawie 11 parametrów, określonych w ww. zbiorze danych.

### 2. Opis implementacji
Implementacja składa się z trzech plików: pliku **wineProcessing.py**, w którym znajdują się funkcje potrzebne do obróbki zbioru danych. Znajduje się tam także sposób podziału win na dwie klasy. Klasa -1 jest przyznawana tym winom, które mają jakość mniejszą bądź równą od zadanej. Klasa 1 jest przyznawana pozostałym winom. Zadaną jakość można modyfikować.

Drugim plikiem jest **testSVM.py**. Znajduje się tam interfejs, dzięki któremu można w łatwy sposób wykonywać testy na algorytmie SVM.

W trzecim pliku - **svm.py**, znajduje się właściwy algorytm. Został on zaimplementowany w klasie SVM. Klasa zawiera następujące metody:

In [ ]:
def learnPolynomialKernel(self):
        cons = {'type':'eq', 'fun': self.constraint}
        bounds = [(0.0, 1/len(self.alpha)*self.lambdaVar*2) for _ in range(len(self.alpha))]
        self.alpha = minimize(self.funDual, self.alpha, (self.polynomialKernel), bounds=(bounds), constraints=cons).x
        self.setB(self.polynomialKernel)

def learnRBFkernel(self):
        cons = {'type':'eq', 'fun': self.constraint}
        bounds = [(0.0, 1/len(self.alpha)*self.lambdaVar*2) for _ in range(len(self.alpha))]
        self.alpha = minimize(self.funDual, self.alpha, (self.RBFkernel), bounds=(bounds), constraints=cons).x
        self.setB(self.RBFkernel)

def constraint(self, alpha):
        output = 0
        for i in range(len(alpha)):
            output += alpha[i] * self.trainingSet[i][1]
        return output

Metody odpowiedzialne za naukę algorytmu z użyciem jądra wielomianowego (w szczególnym przypadku liniowego) lub jądra RBF.

In [ ]:
def polynomialKernel(self, x1, x2):
    return pow(np.dot(x1,x2), self.delta)
    
def RBFkernel(self, x1, x2):
    return exp((-(np.linalg.norm(np.array(x1, dtype=object) - np.array(x2, dtype=object)))**2)/(2*(self.delta**2)))

Metody implementujące jądra. Dla wygody korzystania, hiperparametr delta pełni dwie różne funkcje, w zależności od wykorzystywanego jądra.

In [ ]:
def funDual(self, alpha, kernel):
    firstSum = sum(alpha)
    secondSum = 0
    for i in range(len(alpha)):
        for j in range(len(alpha)):
            i_attributes, i_objectClass = self.trainingSet[i]
            j_attributes, j_objectClass = self.trainingSet[j]
            secondSum += i_objectClass * alpha[i] * j_objectClass * alpha[j] * (kernel(i_attributes, j_attributes))
    return (0.5 * secondSum) - firstSum # -f(x) - maximize

Funkcja celu. Znajduje się w postaci dualnej, gdyż bez niej nie jest możliwe zaimplementowanie funkcji jąder. Jako że w postaci dualnej funkcja powinna być maksymalizowana, a nie minimalizowana, to końcowa różnica jest odwrotna niż jest to podane we wzorze.

In [ ]:
def setB(self, kernel):
    bPoint = None
    for i in range(len(self.alpha)):
        if self.alpha[i] > 0 and self.alpha[i] < 1/(len(self.alpha)*self.lambdaVar*2):
            bPoint = self.trainingSet[i]
    if bPoint is not None:
        bSum = 0
        for j in range(len(self.alpha)):
            bSum += self.alpha[j] * self.trainingSet[j][1] * kernel(self.trainingSet[j][0], bPoint[0])
        self.b = bSum - bPoint[1]

Metoda wyliczająca wartość wyrazu wolnego b.

In [ ]:
def test(self, rbf=False):
    if not rbf:
        kernel = self.polynomialKernel
    else:
        kernel = self.RBFkernel
    positiveResults = 0
    for wine in self.testSet:
        output = 0
        for i in range(len(self.alpha)):
            output += self.alpha[i] * self.trainingSet[i][1] * kernel(self.trainingSet[i][0],wine[0])
            output -= self.b
        wineClass = 1 if output >= 1 else -1
        if wineClass == wine[1]:
            positiveResults += 1
    return (positiveResults/len(self.testSet)* 100)

Metoda oceniająca skuteczność wytrenowanego algorytmu. Dla każdego punktu w zestawie testowym wyliczana jest klasa punktu, a następnie jest to porównywane z faktyczną klasą. Jako wynik zwracana procentowa ilość poprawnych klasyfikacji.

### 3. Eksperymenty

Aby dobrze zbadać działanie algorytmu SVM, postanowiłem przeprowadzić szereg eksperymentów na wielu różnych parametrach. Trening odbywał się zawsze na liczbie 100 prób.  Wyniki w plikach .csv zostały wygenerowane za pomocą następującego skryptu:

In [ ]:
import numpy as np
import csv
from wineProcessing import *
from svm import SVM

def testForQuality(trainingSize, testSize, lambdaVar, delta, delimiter, RBF=True):
    redWines = processWineQuality("winequality-red.csv", delimiter=delimiter)
    whiteWines = processWineQuality("winequality-white.csv", delimiter=delimiter)
    allWines = np.concatenate((redWines, whiteWines))
    np.random.shuffle(allWines)
    trainingSet = allWines[:trainingSize]
    allWines = allWines[trainingSize:]
    testSet = allWines[:testSize]

    svm = SVM(lambdaVar=lambdaVar, trainingSet=trainingSet, testSet=testSet, delta=delta, startingAlpha=np.array([1]*(len(trainingSet))))
    if RBF:
        svm.learnRBFkernel()
        return svm.test(rbf=True)
    else:
        svm.learnPolynomialKernel()
        return svm.test(rbf=False)

if __name__ == "__main__":

    testSize = 6000
    delimiters = [3,5,7]
    lambdas = [0.01,0.1,1]
    deltas = [1,10,100]
    trainingSizes= [100]
    generateRBF = False
    generatePolynomial = False
    generateLinear = True

    if generateRBF:
        with open("rbf.csv", 'w') as file:

            writer = csv.DictWriter(file, fieldnames=["delimiter", "lambda", "delta", "training set size", "effectiveness"])
            writer.writeheader()

            for lambdaVar in lambdas:
                for delta in deltas:
                    for trainingSize in trainingSizes:
                        for delimiter in delimiters:
                            percent = testForQuality(trainingSize=trainingSize, testSize=6000, lambdaVar=0.01, delta=delta, delimiter=delimiter, RBF=True)
                            writer.writerow({"delimiter":delimiter, "lambda":lambdaVar, "delta":delta, "training set size":trainingSize, "effectiveness":round(percent, 2)})
    
    if generatePolynomial:
        with open("polynomial.csv", 'w') as file:

            writer = csv.DictWriter(file, fieldnames=["delimiter", "lambda", "delta", "training set size", "effectiveness"])
            writer.writeheader()

            for lambdaVar in lambdas:
                for delta in deltas:
                    for trainingSize in trainingSizes:
                        for delimiter in delimiters:
                            percent = testForQuality(trainingSize=trainingSize, testSize=6000, lambdaVar=0.01, delta=delta, delimiter=delimiter, RBF=False)
                            writer.writerow({"delimiter":delimiter, "lambda":lambdaVar, "delta":delta, "training set size":trainingSize, "effectiveness":round(percent, 2)})
    
    if generateLinear:
        with open("linear.csv", 'w') as file:

            writer = csv.DictWriter(file, fieldnames=["delimiter", "lambda", "delta", "training set size", "effectiveness"])
            writer.writeheader()

            for lambdaVar in lambdas:
                for trainingSize in trainingSizes:
                    for delimiter in delimiters:
                        percent = testForQuality(trainingSize=trainingSize, testSize=6000, lambdaVar=0.01, delta=1, delimiter=delimiter, RBF=False)
                        writer.writerow({"delimiter":delimiter, "lambda":lambdaVar, "delta":1, "training set size":trainingSize, "effectiveness":round(percent, 2)})


Wyniki znajdują się w wygenerowanych plikach CSV. Istnieje opcja ich zwizualizowania przy pomocy następującej funkcji:

In [6]:
import csv

def generateResultsFromCsv(filename):
    with open(filename, 'r') as file:
        reader = csv.DictReader(file)
        fieldnames = reader.fieldnames
        for field in fieldnames[:-1]:
            effectivenessField = {}
            for row in reader:
                if row[field] not in effectivenessField:
                    effectivenessField[row[field]] = list()
                effectivenessField[row[field]].append(float(row['effectiveness']))
            print("\n", field, "\n")
            for key, value in effectivenessField.items():
                print(f"{key} - effectiveness {round(sum(value)/len(value),2)}%")
            file.seek(0)
            file.readline()
        effectivenessList = []
        for row in reader:
            effectivenessList.append(float(row['effectiveness']))
        print(f"\nGeneral effectiveness: {round(sum(effectivenessList)/len(effectivenessList),2)}%")

if __name__ == "__main__":
    print("Linear kernel: \n")
    generateResultsFromCsv("linear.csv")
    print("\nRBF kernel: \n")
    generateResultsFromCsv("rbf.csv")
    print("\nPolynomial kernel: \n")
    generateResultsFromCsv("polynomial.csv")

Linear kernel: 


 delimiter 

3 - effectiveness 99.54%
5 - effectiveness 63.44%
7 - effectiveness 96.95%

 lambda 

0.01 - effectiveness 86.64%
0.1 - effectiveness 86.61%
1 - effectiveness 86.68%

 delta 

1 - effectiveness 86.64%

 training set size 

100 - effectiveness 86.64%

General effectiveness: 86.64%

RBF kernel: 


 delimiter 

3 - effectiveness 77.53%
5 - effectiveness 57.39%
7 - effectiveness 96.94%

 lambda 

0.01 - effectiveness 72.6%
0.1 - effectiveness 75.62%
1 - effectiveness 83.63%

 delta 

1 - effectiveness 75.62%
10 - effectiveness 83.59%
100 - effectiveness 72.65%

 training set size 

100 - effectiveness 77.28%

General effectiveness: 77.28%

Polynomial kernel: 


 delimiter 

3 - effectiveness 43.59%
5 - effectiveness 51.49%
7 - effectiveness 74.94%

 lambda 

0.01 - effectiveness 53.85%
0.1 - effectiveness 61.42%
1 - effectiveness 54.75%

 delta 

1 - effectiveness 86.64%
10 - effectiveness 38.68%
100 - effectiveness 44.7%

 training set size 

100 - effective

### 4. Wnioski

Powyższe eksperymenty doprowadzają nas do paru wniosków.

Po pierwsze, ogółem najlepszym okazało się być jądro liniowe, ze skutecznością na poziomie ponad 80%. Oznacza to że dane są generalnie liniowo separowalne w dużym stopniu.

Po drugie, najlepszy możliwy parametr lambda zmienia się wraz z używanym jądrem. Dla jądra liniowego odpowiednia jest dowolna z podanych wartości, dla jądra RBF duża wartość, a dla jądra wielomianowego - średnia wartość (0.01). Jest to oznaka tego, że dla jądra RBF najlepsza jest twarda, "ciasna" granica, nie dopuszczająca złych klasyfikacji. Dla jądra liniowego margines jest na tyle duży że zmiana parametru lambda nie dokonuje większych zmian, a dla jądra wielomianowego dobry jest średni margines.

Po trzecie, dla jądra wielomianowego parametr delta sprawdza się najlepiej, gdy jest możliwie mały. To oznacza, że problem jest w dużej mierze liniowo separowalny.

Warto zwrócić uwagę na wartości graniczne. Pomijając jądro wielomianowe, dużo prościej jest sklasyfikować obiekty znajdujące się na końcach skali. Jest to uzasadnione. Wartości bliżej końca skali jest dużo mniej niż tych średnich, a więc prościej jest też je oddzielić od reszty danych.

### 5. Podsumowanie

Algorytm SVM jest jak najbardziej dobrym klasyfikatorem, choć należy uważać na odpowiednie dobranie hiperparametrów.
Niestety jego wadą jest długi czas działania przy większych próbach. Aby wygenerować powyższe wyniki, musiałem poświęcić około 30 minut czasu pracy komputera.